In [63]:
import pymysql
from datetime import datetime as dt
import pandas as pd

In [76]:
class PortfolioTracker():
    def __init__(self):
        self.conn = pymysql.connect(
            host='localhost',
            user='root', 
            password = "password",
            port=6603,
            db='PortfolioTracker',
            )

    def find_asset(self, idcustom):
        c = self.conn.cursor()
        sql = 'Select idasset from assets where idcustom=%s'
        c.execute(sql, (idcustom,) )
        idasset = c.fetchone()
        return idasset

    def find_portfolio(self, idcustom):
        c = self.conn.cursor()
        sql = 'Select idportfolio from portfolios where idcustom=%s'
        c.execute(sql, (idcustom,) )
        idasset = c.fetchone()
        return idasset

    def create_asset(self, idcustom, name):
        c = self.conn.cursor()
        sql = 'INSERT INTO assets (idcustom, name) VALUES (%s, %s)'
        c.execute(sql, (idcustom, name,))
        self.conn.commit()
        return self.find_asset(idcustom)

    def create_portfolio(self, idcustom, name):
        c = self.conn.cursor()
        sql = 'INSERT INTO portfolios (idcustom, name) VALUES (%s, %s)'
        c.execute(sql, (idcustom, name,))
        self.conn.commit()
        return self.find_portfolio(idcustom)
    
    def load_trade(self, idportfolio, idasset, date, qty, price):
        c = self.conn.cursor()
        print(date)
        sql = 'INSERT INTO trades (idportfolio, idasset, date, qty, price) VALUES (%s, %s, %s, %s, %s)'
        c.execute(sql, (idportfolio, idasset, dt.strptime(date,'%d/%m/%Y'), qty, price))
        self.conn.commit()
        return 0
    
    def load_trade_from_file(self, path):
        df = pd.read_csv(path)
        for val in df.values:
            #find asset/portfolio
            pname = self.find_portfolio(val[0])
            aname = self.find_asset(val[1])
            #create if not found
            if pname is None:
                pname = self.create_portfolio(val[0], val[0])
            if aname is None:
                aname = self.create_asset(val[1], val[1])
            #load trade
            self.load_trade(pname, aname, *val[2:])
    
#TODO: create price loader, create functions to request trades and valuations for date/portfolio 


In [78]:
myp = PortfolioTracker()
myp.load_trade_from_file('trade_file.csv')

16/04/2022
16/04/2022
16/04/2022
16/04/2022
16/04/2022
16/04/2022
16/04/2022


In [70]:
for val in df.values:
    a = myp.find_asset('testa')
    p = myp.find_portfolio('testp')
    myp.load_trade(p, a, *val[2:])

16/04/2022
16/04/2022
16/04/2022
16/04/2022
16/04/2022
16/04/2022


In [61]:
dt.strptime('16/04/2022','%d/%m/%Y')

datetime.datetime(2022, 4, 16, 0, 0)